In [190]:
# !cat coords.csv | awk -vFPAT='([^,]*)|("[^"]+")' -vOFS=, '{if($1==981 || $1==982) print }' > sample_981_982.csv

In [214]:
import pandas as pd
import numpy as np
from shapely.geometry import LineString, Point, MultiPoint
import uuid
from geographiclib.geodesic import Geodesic
from haversine import haversine,Unit
from shapely.geometry import Point
from shapely import ops
import math

from sklearn.neighbors import KDTree
import numpy as np


Compare all groups --> Identify spatial intersections.

In [ ]:

data = pd.read_csv('coords.csv',header=None, names=['id','time','lat','lon','preds','c1','c2','linestring'])
data = data.drop(columns=['preds','c1','c2','linestring'])
def calc_speed(data):
    data['lag_lon']= data['lon'].shift(1)
    data['lag_lat']= data['lat'].shift(1)
    data['ddt'] = data['time'].diff().fillna(method='backfill')/1000
    data['course'] = data.apply(lambda x: Geodesic.WGS84.Inverse(x.lag_lat,x.lag_lon,x.lat,x.lon)['azi2'],axis=1).fillna(method='backfill')
    data['speed'] = data.apply(lambda x: ((Geodesic.WGS84.Inverse(x.lag_lat,x.lag_lon,x.lat,x.lon)['s12'])/x.ddt)*1.9438452 ,axis=1).fillna(method='backfill')
    data['course'] = data['course'].apply(lambda x: (x+360)%360)
    data['heading']= data['course']

# data = data.rename(columns={'lat':'lon', 'lon':'lat'})
data[['lon','lat']] = data[['lat','lon']]
groups = data.sort_values(['id','time'],ascending=True).groupby(by='id')
data_all = pd.DataFrame(columns=['id','time','lon','lat','coll_id'])
cnt=0
collisions_df = pd.DataFrame(columns=['id1','id2','x','y','t'])


def get_closest(data, x, y):
    distances = np.sqrt((data['lon'] - x)**2 + (data['lat'] - y)**2)
    closest_idx = distances.idxmin()
    return distances[closest_idx],closest_idx 

for g1 in groups:
    coordinates1 = list(zip(g1[1]['lat'], g1[1]['lon']))
    if len(coordinates1)>1:
        line1 = LineString(coordinates=coordinates1)
    else:
        continue
    for g2 in groups:
        if g1[0] > g2[0]:
            coordinates2 = list(zip(g2[1]['lat'], g2[1]['lon']))
            if len(coordinates2)>1:
                line2 = LineString(coordinates=coordinates2)
                if line1.intersects(line2) and g1[1].shape[0]>2 and g2[1].shape[0]>2:
                    cnt+=1
                    inter = line1.intersection(line2)
                    if inter.geom_type == 'Point':
                        x, y = inter.x, inter.y
                    elif inter.geom_type == 'MultiPoint':
                        point = inter.geoms[0]
                        x, y = point.x, point.y
                    
                    # Assign collision IDs
                    coll_id = cnt #uuid.uuid4()
                    g1[1]['coll_id'] = coll_id
                    g2[1]['coll_id'] = coll_id

                    # Calculate speeds, Knots/sec
                    calc_speed(g1[1])
                    calc_speed(g2[1])
                    
                    
                    dist_a, a_index  = get_closest(coordinates1,x,y)
                    dist_c, c_index  = get_closest(coordinates2,x,y)
                    kdt1 = KDTree(coordinates1)
                    nearest_dist, nearest_ind  = kdt1.query([[x,y]],k=2)
                    a_index1 = nearest_ind[0][0]
                    a_index2 = nearest_ind[0][1]
                    a_index = a_index1 if g1[1]['time'].iloc[a_index1] <  g1[1]['time'].iloc[a_index2] else a_index2
                    
                    
                    kdt2 = KDTree(coordinates2)
                    nearest_dist2, nearest_ind2  = kdt2.query([[x,y]],k=2)
                    c_index1 = nearest_ind2[0][0]
                    c_index2 = nearest_ind2[0][1]
                    c_index = c_index1 if g2[1]['time'].iloc[c_index1] <  g2[1]['time'].iloc[c_index2] else c_index2
                    
                    # Calculate time aligment
                    crush_point = Point(x,y)
                    a = Point(g1[1]['lat'].iloc[a_index], g1[1]['lon'].iloc[a_index])
                    # Meters
                    ax = Geodesic.WGS84.Inverse(a.y,a.x, crush_point.y ,crush_point.x)['s12'] 
                    # Seconds, speed m/s
                    t_crush1 = ax / (g1[1]['speed'].iloc[a_index]/ 1.9438452 )
                    # Meters 
                    c = Point(g2[1]['lat'].iloc[c_index], g2[1]['lon'].iloc[c_index])
                    cx = Geodesic.WGS84.Inverse(c.y,c.x, crush_point.y ,crush_point.x)['s12']
                    # Meters / m/s
                    t_crush2 = cx / (g2[1]['speed'].iloc[c_index] / 1.9438452) 
                    # dt of ship2
                   
                    # Offset
                    offset = int((g1[1]['time'].iloc[a_index] + t_crush1*1000 ) - ((g2[1]['time'].iloc[c_index]) + t_crush2*1000))
                    for i in range(g2[1].shape[0]):
                        g2[1].at[i, 'time'] = g2[1]['time'].iloc[i]
                        g2[1].at[i, 'new_time'] = g2[1]['time'].iloc[i] - 2*offset
                    
                     
                    g2[1]['time'] =  g2[1]['time'] 
                    g2[1]['new_time'] =  g2[1]['time'] + offset

                    g1[1]['new_time'] = g1[1]['time']
                    temp = pd.DataFrame(data={'id1':g1[0], 'id2':g2[0],'x':x,'y':y,'t':0}, index=[0])
                    collisions_df = pd.concat([collisions_df, temp]).reset_index(drop=True)
                    data_all = pd.concat([data_all,g1[1],g2[1]])
            else:
                continue



Time align pair per spatial intersection.

In [ ]:
def update_row(data):
    data['lag_lon']= data['lon'].shift(1)
    data['lag_lat']= data['lat'].shift(1)
    data['ddt'] = data['time'].diff().fillna(method='backfill')/1000
    data['course'] = data.apply(lambda x: Geodesic.WGS84.Inverse(x.lag_lat,x.lag_lon,x.lat,x.lon)['azi2'],axis=1).fillna(method='backfill')
    data['speed'] = data.apply(lambda x: ((Geodesic.WGS84.Inverse(x.lag_lat,x.lag_lon,x.lat,x.lon)['s12'])/x.ddt)*1.9438452 ,axis=1).fillna(method='backfill')
    data['course'] = data['course'].apply(lambda x: (x+360)%360)
    data['heading']= data['course']        
            
data_all = pd.DataFrame(columns=['id','time','lon','lat','coll_id'])
collisions_copy = pd.DataFrame(columns=['id1','id2','x','y','t','coll_id'])
cnt=0

iter_offset = 0 



for row in collisions_df.itertuples():
    cnt+=1
    x= row.x
    y=row.y
    
    g1 = groups.get_group(int(row.id1)).reset_index(drop=True).copy()
    g2 = groups.get_group(int(row.id2)).reset_index(drop=True).copy()
    
    g1['coll_id'] = cnt
    g2['coll_id'] = cnt
    
    coordinates1 = list(zip(g1['lat'], g1['lon']))
    coordinates2 = list(zip(g2['lat'], g2['lon']))
    
    line1 = LineString(coordinates=coordinates1)
    line2 = LineString(coordinates=coordinates2)
    
    update_row(g1)
    update_row(g2)
    crush_point = Point(x,y)

    a_index = 0
    for i in range(g1.shape[0]-1):
        c1 = list(zip(g1['lat'].iloc[i:i+2], g1['lon'].iloc[i:i+2]))
        l1 = LineString(coordinates=c1)
        if l1.intersects(crush_point.buffer(0.001)):
            a_index = i
            # break
    c_index = 0
    for j in range(g2.shape[0]-1):
        c2 = list(zip(g2['lat'].iloc[j:j+2], g2['lon'].iloc[j:j+2]))
        l2 = LineString(coordinates=c2)
        if l2.intersects(crush_point.buffer(0.001)):
            c_index = j
            # break
    
    kdt1 = KDTree(coordinates1)
    nearest_dist, nearest_ind  = kdt1.query([[x,y]],k=2)
    a_index1 = nearest_ind[0][0]
    a_index2 = nearest_ind[0][1]
    a_index = a_index1 if g1['time'].iloc[a_index1] <  g1['time'].iloc[a_index2] else a_index2
    
    
    kdt2 = KDTree(coordinates2)
    nearest_dist2, nearest_ind2  = kdt2.query([[x,y]],k=2)
    c_index1 = nearest_ind2[0][0]
    c_index2 = nearest_ind2[0][1]
    c_index = c_index1 if g2['time'].iloc[c_index1] <  g2['time'].iloc[c_index2] else c_index2
    
    # Calculate time aligment
    a = Point(g1['lat'].iloc[a_index], g1['lon'].iloc[a_index])
    # Meters
    ax = Geodesic.WGS84.Inverse(a.y,a.x, crush_point.y ,crush_point.x)['s12'] 
    # Seconds, speed m/s
    t_crush1 = ax / (g1['speed'].iloc[a_index]/ 1.9438452 )
    # Meters 
    c = Point(g2['lat'].iloc[c_index], g2['lon'].iloc[c_index])
    cx = Geodesic.WGS84.Inverse(c.y,c.x, crush_point.y ,crush_point.x)['s12']
    # Meters / m/s
    t_crush2 = cx / (g2['speed'].iloc[c_index] / 1.9438452) 
    # dt of ship2
    
    # Offset
    offset = int((g1['time'].iloc[a_index] + t_crush1*1000 ) - ((g2['time'].iloc[c_index]) + t_crush2*1000))
    
    
    
    crush_time =  g2['time'].iloc[c_index] +iter_offset+ offset
    g2['time'] = g2['time'] +iter_offset+ offset
    g1['time'] = g1['time'] +iter_offset
    
    print(g1.id, g2.id)
    g1.id = cnt*10 +1 
    g2.id = cnt*10 +2
    print(g1.id, g2.id)
    
    t1 = g1['time'].iloc[a_index] 

    t2 = g2['time'].iloc[c_index]
    # print(10*'-')
    # print(f"{g1.id.iloc[0]}, {g2.id.iloc[0]}, {abs(t2-t1)}")
    
    # print(10*'-')

    data_all = pd.concat([data_all,g1,g2])
    
    iter_offset+=2*3600*1000
    
    collisions_copy = pd.concat([collisions_copy,  \
                                  pd.DataFrame(data={'id1':cnt*10 +1, 'id2':cnt*10 +2,'x':x,'y':y,'t':crush_time,\
                                               'coll_id':cnt, 't1':t1, 't2':t2, 't1_crush':t_crush1, 't2_crush':t_crush2, \
                                                'crush1':a_index,'crush1_total':g1.shape[0], 'crush2':c_index,'crush2_total':g2.shape[0]}, index=[0])]).reset_index(drop=True)
    
collisions_copy['id1'] = collisions_copy['id1'].astype('Int64')
collisions_copy['id2'] = collisions_copy['id2'].astype('Int64')

In [ ]:
collisions_copy['expected_diff_nn1'] = abs(((collisions_copy['t1']-collisions_copy['t2'])/60000))
collisions_copy['expected_diff_crush'] = abs(((collisions_copy['t1_crush']-collisions_copy['t2_crush'])/60))
collisions_copy['missaligment'] = abs(collisions_copy['expected_diff_nn1'] - collisions_copy['expected_diff_crush'])
collisions_copy

### Discard events with under N points

In [ ]:
# Discard events with less than 10 points from both 
a = data_all.groupby(by='id')['lon'].count().sort_values()
a = a > 10
a = a[a==True].index
data_all = data_all[data_all['id'].isin(a)].reset_index(drop=True)
collisions_copy = collisions_copy[(collisions_copy['id1'].isin(a))&(collisions_copy['id2'].isin(a))].reset_index(drop=True)
# Discard events that collision appear in the first 3 points
collisions_copy = collisions_copy[(collisions_copy['crush1']>3)&(collisions_copy['crush2']>3)]
cols = pd.concat([collisions_copy['id1'], collisions_copy['id2']])
data_all = data_all[data_all['id'].isin(cols)].reset_index(drop=True)
collisions_copy

### Set threshold here

In [ ]:
# minutes
threshold = 30
collisions_copy_ids = collisions_copy[collisions_copy['expected_diff_nn1']< threshold ][['id1','id2']]
d = pd.concat([collisions_copy_ids['id1'], collisions_copy_ids['id2']])
data_all = data_all[data_all['id'].isin(d)].reset_index(drop=True)


# Save
data_output = pd.DataFrame(columns=['station','mmsi','t','lon','lat','heading','course','speed','status'])
data_all['station'] = 1
data_all['status'] = 1

data_output = data_all[['station','id','time','lon','lat','heading','course','speed','status']]
data_output = data_output.rename(columns={
    'id':'mmsi',
    'time':'t'
})
data_output = data_output.astype({'station':'int',
                'mmsi':'int',
                # 't':"int",
                "lon":"float",
                "lat":"float",
                "heading":"int",
                "course":"int",
                "speed":"int",
                "status":"int"})
data_output = data_output.fillna(value=0).sort_values(by='t', ascending=True).drop_duplicates()
data_output.to_csv('collision_sample.csv',index=False)
collisions_copy[['x','y']] = collisions_copy[['y','x']]
collisions_copy.to_csv('results/collision_true.csv', index=False)